In [11]:
import torch
import os
import torch.nn as nn
import numpy as np
from torch.nn.utils import clip_grad_norm

In [12]:

class Dictionary(object):
    def __init__(self):
        self.word2idx = {}
        self.idx2word = {}
        self.idx = 0

    def add_word(self, word):
        if word not in self.word2idx:
            self.word2idx[word] = self.idx
            self.idx2word[self.idx] = word
            self.idx += 1
            
    def __len__(self):
        return len(self.word2idx)

In [13]:
class TextProcess(object):
    
    def __init__(self):
        self.dictionary = Dictionary()

    def get_data(self, path, batch_size=20):
        with open(path, 'r') as f:
            tokens = 0
            for line in f:
                words = line.split() + ['<eos>']
                tokens += len(words)
                for word in words: 
                    self.dictionary.add_word(word)  
        #Create a 1-D tensor that contains the index of all the words in the file
        rep_tensor = torch.LongTensor(tokens)
        index = 0
        with open(path, 'r') as f:
            for line in f:
                words = line.split() + ['<eos>']
                for word in words:
                    rep_tensor[index] = self.dictionary.word2idx[word]
                    index += 1
        #Find out how many batches we need            
        num_batches = rep_tensor.shape[0] // batch_size     
        #Remove the remainder (Filter out the ones that don't fit)
        rep_tensor = rep_tensor[:num_batches*batch_size]
        # return (batch_size,num_batches)
        rep_tensor = rep_tensor.view(batch_size, -1)
        return rep_tensor

In [14]:
embed_size = 128    #Input features to the LSTM
hidden_size = 1024  #Number of LSTM units
num_layers = 1
num_epochs = 20
batch_size = 20
timesteps = 30
learning_rate = 0.002

corpus = TextProcess()
rep_tensor = corpus.get_data('earnest.txt', batch_size)
#rep_tensor is the tensor that contains the index of all the words. Each row contains 1659 words by default 
print(rep_tensor.shape)

vocab_size = len(corpus.dictionary)
print(vocab_size)

num_batches = rep_tensor.shape[1] // timesteps
print(num_batches)

torch.Size([20, 1361])
4930
45


In [15]:

class TextGenerator(nn.Module):
    
    def __init__(self, vocab_size, embed_size, hidden_size, num_layers):
        super(TextGenerator, self).__init__()
        self.embed = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True)
        self.linear = nn.Linear(hidden_size, vocab_size)

    def forward(self, x, h):
        # Perform Word Embedding 
        x = self.embed(x)
        #Reshape the input tensor
        #x = x.view(batch_size,timesteps,embed_size)
        out, (h, c) = self.lstm(x, h)
        # Reshape the output from (samples,timesteps,output_features) to a shape appropriate for the FC layer 
        # (batch_size*timesteps, hidden_size)
        out = out.reshape(out.size(0)*out.size(1), out.size(2))
        # Decode hidden states of all time steps
        out = self.linear(out)
        return out, (h, c)

In [16]:
model = TextGenerator(vocab_size, embed_size, hidden_size, num_layers)

In [17]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [18]:
def detach(states):
    """
If we have a tensor z,'z.detach()' returns a tensor that shares the same storage
as 'z', but with the computation history forgotten. It doesn't know anything
about how it was computed. In other words, we have broken the tensor z away from its past history
Here, we want to perform truncated Backpropagation
TBPTT splits the 1,000-long sequence into 50 sequences (say) each of length 20 and treats each sequence of length 20 as 
a separate training case. This is a sensible approach that can work well in practice, but it is blind to temporal 
dependencies that span more than 20 timesteps.
    """
    return [state.detach() for state in states]

In [19]:
for epoch in range(num_epochs):
    # Set initial hidden and cell states
    states = (torch.zeros(num_layers, batch_size, hidden_size),
              torch.zeros(num_layers, batch_size, hidden_size))

    for i in range(0, rep_tensor.size(1) - timesteps, timesteps):
        # Get mini-batch inputs and targets
        inputs = rep_tensor[:, i:i+timesteps]  
        targets = rep_tensor[:, (i+1):(i+1)+timesteps]
        
        outputs,_ = model(inputs, states)
        loss = loss_fn(outputs, targets.reshape(-1))

        model.zero_grad()
        loss.backward()
        #Perform Gradient Clipping. clip_value (float or int) is the maximum allowed value of the gradients 
        #The gradients are clipped in the range [-clip_value, clip_value]. This is to prevent the exploding gradient problem
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()
              
        step = (i+1) // timesteps
        if step % 100 == 0:
            print ('Epoch [{}/{}], Loss: {:.4f}'
                   .format(epoch+1, num_epochs, loss.item()))

<ipython-input-19-69c25f07fb5f>:18: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  clip_grad_norm(model.parameters(), 0.5)


Epoch [1/20], Loss: 8.5037
Epoch [2/20], Loss: 6.0226
Epoch [3/20], Loss: 5.2255
Epoch [4/20], Loss: 4.6082
Epoch [5/20], Loss: 4.1354
Epoch [6/20], Loss: 3.5891
Epoch [7/20], Loss: 3.0369
Epoch [8/20], Loss: 2.6220
Epoch [9/20], Loss: 2.3021
Epoch [10/20], Loss: 2.0140
Epoch [11/20], Loss: 1.6751
Epoch [12/20], Loss: 1.4020
Epoch [13/20], Loss: 1.1175
Epoch [14/20], Loss: 0.8251
Epoch [15/20], Loss: 0.5717
Epoch [16/20], Loss: 0.3829
Epoch [17/20], Loss: 0.2416
Epoch [18/20], Loss: 0.1537
Epoch [19/20], Loss: 0.1108
Epoch [20/20], Loss: 0.0866


In [20]:
# Test the model
with torch.no_grad():
    with open('results.txt', 'w') as f:
        # Set intial hidden ane cell states
        state = (torch.zeros(num_layers, 1, hidden_size),
                 torch.zeros(num_layers, 1, hidden_size))
        # Select one word id randomly and convert it to shape (1,1)
        input = torch.randint(0,vocab_size, (1,)).long().unsqueeze(1)

        for i in range(500):
            output, _ = model(input, state)
            print(output.shape)
            # Sample a word id from the exponential of the output 
            prob = output.exp()
            word_id = torch.multinomial(prob, num_samples=1).item()
            print(word_id)
            # Replace the input with sampled word id for the next time step
            input.fill_(word_id)

            # Write the results to file
            word = corpus.dictionary.idx2word[word_id]
            word = '\n' if word == '<eos>' else word + ' '
            f.write(word)
            
            if (i+1) % 100 == 0:
                print('Sampled [{}/{}] words and save to {}'.format(i+1, 500, 'results.txt'))

torch.Size([1, 4930])
17
torch.Size([1, 4930])
70
torch.Size([1, 4930])
182
torch.Size([1, 4930])
49
torch.Size([1, 4930])
16
torch.Size([1, 4930])
11
torch.Size([1, 4930])
3631
torch.Size([1, 4930])
11
torch.Size([1, 4930])
496
torch.Size([1, 4930])
25
torch.Size([1, 4930])
17
torch.Size([1, 4930])
11
torch.Size([1, 4930])
496
torch.Size([1, 4930])
25
torch.Size([1, 4930])
183
torch.Size([1, 4930])
702
torch.Size([1, 4930])
68
torch.Size([1, 4930])
69
torch.Size([1, 4930])
11
torch.Size([1, 4930])
11
torch.Size([1, 4930])
981
torch.Size([1, 4930])
419
torch.Size([1, 4930])
11
torch.Size([1, 4930])
496
torch.Size([1, 4930])
25
torch.Size([1, 4930])
183
torch.Size([1, 4930])
195
torch.Size([1, 4930])
25
torch.Size([1, 4930])
1159
torch.Size([1, 4930])
426
torch.Size([1, 4930])
3961
torch.Size([1, 4930])
3656
torch.Size([1, 4930])
204
torch.Size([1, 4930])
205
torch.Size([1, 4930])
476
torch.Size([1, 4930])
962
torch.Size([1, 4930])
1346
torch.Size([1, 4930])
2973
torch.Size([1, 4930])
1